#  Percentage Change - Chordia Model

Measuring changes in individual stock liquidity regressed on changes in Market Liquidity using a model inspired by Chordia et al (2000).

The model is as follows: DLjt = @ + DLMt + DLM(t-1) + DLM(t+1) + e

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

## Importing the initial data set

In [2]:
df=pd.read_excel("BBREIT-1.07.xlsx",sheet_name="PROPORTIONAL_BID_ASK_SPREAD", index_col="Dates")
df.dropna(axis=1,inplace=True)

In [3]:
df.dropna(axis=1,inplace=True)


## Defining thersholds to omit extremal values

In [4]:
q =df.quantile(0.9999)
b =df.quantile(0.0001)

In [5]:
df=df[(df < q) & (df > b)]



## Replacing resulting NaN values with the value of the previous period

In [6]:
df.fillna(method="bfill",inplace=True)

In [7]:
df.isnull().values.sum()

153

In [8]:
df.dropna(axis=0,inplace=True)

In [9]:
df

,AMT US Equity,PLD US Equity,CCI US Equity,EQIX US Equity,PSA US Equity,O US Equity,SBAC US Equity,WELL US Equity,VICI US Equity,DLR US Equity,...,CLPR US Equity,VOC US Equity,FREVS US Equity,AAIC US Equity,SELF US Equity,PW US Equity,SOHO US Equity,CORR US Equity,PEI US Equity,WHLR US Equity
Dates,,,,,,,,,,,,,,,,,,,,,
2020-08-04,0.000192,0.000095,0.000060,0.000895,0.000050,0.000162,0.000870,0.000188,0.000440,0.000062,...,0.003130,0.005602,0.033898,0.003752,0.005236,0.007507,0.013857,0.001082,0.009569,0.016713
2020-08-05,0.000353,0.000095,0.000365,0.000529,0.000508,0.000963,0.001208,0.002260,0.000447,0.000125,...,0.003175,0.011299,0.033898,0.003591,0.013072,0.057373,0.009217,0.001053,0.009569,0.010753
2020-08-06,0.000744,0.000095,0.000366,0.000493,0.000765,0.000161,0.000228,0.000177,0.000440,0.000064,...,0.001537,0.011905,0.033898,0.003656,0.031088,0.017939,0.017778,0.006270,0.009390,0.005038
2020-08-07,0.000039,0.000285,0.000059,0.000657,0.000050,0.000477,0.000546,0.000175,0.001748,0.000063,...,0.002967,0.040115,0.033898,0.003604,0.002594,0.021964,0.008696,0.000984,0.009479,0.013423
2020-08-10,0.000671,0.000385,0.000181,0.000447,0.000049,0.000319,0.000499,0.000174,0.000432,0.000379,...,0.004264,0.022599,0.033898,0.003527,0.012804,0.015131,0.012685,0.000956,0.007968,0.025316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-19,0.000040,0.000089,0.000122,0.000160,0.000032,0.000156,0.000481,0.000140,0.000298,0.000090,...,0.003706,0.004422,0.052946,0.003210,0.007105,0.034568,0.009479,0.013921,0.029703,0.024845
2022-09-20,0.000042,0.000360,0.000372,0.000292,0.000033,0.000158,0.000529,0.000144,0.000303,0.000091,...,0.002519,0.008811,0.032749,0.003231,0.016231,0.009125,0.009524,0.009524,0.015190,0.013514
2022-09-21,0.000299,0.000092,0.000505,0.000661,0.000033,0.000160,0.000666,0.000145,0.000312,0.000092,...,0.001249,0.007168,0.014556,0.003252,0.024691,0.004158,0.009804,0.014528,0.014184,0.006826


## Transforming data to Percentage Change

In [10]:
for i in df:
    df[i]= (df[i]-df[i].shift(1))/df[i].shift(1)
    
df

,AMT US Equity,PLD US Equity,CCI US Equity,EQIX US Equity,PSA US Equity,O US Equity,SBAC US Equity,WELL US Equity,VICI US Equity,DLR US Equity,...,CLPR US Equity,VOC US Equity,FREVS US Equity,AAIC US Equity,SELF US Equity,PW US Equity,SOHO US Equity,CORR US Equity,PEI US Equity,WHLR US Equity
Dates,,,,,,,,,,,,,,,,,,,,,
2020-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-05,0.835260,0.006578,5.049407,-0.408968,9.235482,4.945951,0.388183,11.046336,0.014749,1.018861,...,0.014286,1.016949,0.000000,-0.043088,1.496732,6.642044,-0.334869,-0.026330,0.000000,-0.356631
2020-08-06,1.109706,0.000191,0.003966,-0.068573,0.506314,-0.832576,-0.811488,-0.921578,-0.014534,-0.491671,...,-0.515757,0.053571,0.000000,0.018282,1.378238,-0.687330,0.928889,4.952978,-0.018779,-0.531486
2020-08-07,-0.947777,1.993720,-0.837609,0.334083,-0.934417,1.961574,1.397442,-0.013293,2.969406,-0.010776,...,0.930267,2.369628,0.000000,-0.014414,-0.916559,0.224406,-0.510870,-0.843089,0.009479,1.664430
2020-08-10,16.278562,0.350228,2.036463,-0.319306,-0.016050,-0.331659,-0.086663,-0.004700,-0.752862,5.014213,...,0.437100,-0.436642,0.000000,-0.021164,3.935980,-0.311116,0.458774,-0.028667,-0.159363,0.886076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-19,-0.933778,0.007980,-0.001224,-0.002012,-0.922934,0.005315,6.502527,-0.489088,-0.004770,0.007648,...,-0.499691,-0.495210,2.382884,0.003210,-0.330965,2.791470,1.071090,0.000000,-0.348880,0.320911
2022-09-20,0.040272,3.065429,2.036603,0.828316,0.020308,0.013947,0.099494,0.024294,0.017286,0.012757,...,-0.320319,0.992658,-0.381463,0.006462,1.284491,-0.736031,0.004762,-0.315873,-0.488608,-0.456081
2022-09-21,6.112722,-0.744461,0.359520,1.263225,0.015113,0.008794,0.258282,0.008554,0.029664,0.008917,...,-0.504060,-0.186380,-0.555530,0.006504,0.521262,-0.544321,0.029412,0.525424,-0.066194,-0.494881


In [11]:
df.dropna(axis=0,inplace=True)

### Defining sum-variable

In [12]:
df["Sum"]=df.sum(axis=1)

## Model

In [13]:
# Defining lists to be filled with relevant values
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    
    df["M"]=((df["Sum"]-df[i])/157) # Cross Sectional Mean Variable
    df["Mlag"]=df["M"].shift(1) # Lag Variable
    df["Mlead"]=df["M"].shift(-1) # Lead variable
    df.replace([np.inf], np.nan, inplace=True) # Replacing possible inf values with NaN
    df=df.fillna(0) # Filling NaN values with zero
        
        
    # Model 
    y = df[i]
    x = df[["M","Mlag","Mlead"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","Mlag","Mlead"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     4.536
Date:                Tue, 22 Nov 2022   Prob (F-statistic):            0.00375
Time:                        14:14:10   Log-Likelihood:                -1734.8
No. Observations:                 558   AIC:                             3478.
Df Residuals:                     554   BIC:                             3495.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3210      0.509      2.597      0.0

(8.32002289946346, 0.03984070983308684, 2.7951370969316494, 0.039660494135778476)
2.2266542762062222
                            OLS Regression Results                            
Dep. Variable:          DRE US Equity   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     8.998
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           7.93e-06
Time:                        14:14:11   Log-Likelihood:                -740.98
No. Observations:                 558   AIC:                             1490.
Df Residuals:                     554   BIC:                             1507.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

(2.9826210982457817, 0.3943124296195615, 0.9923845938529017, 0.39598428635520366)
2.529044894957306
                            OLS Regression Results                            
Dep. Variable:         REXR US Equity   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     17.35
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           8.87e-11
Time:                        14:14:11   Log-Likelihood:                -1148.7
No. Observations:                 558   AIC:                             2305.
Df Residuals:                     554   BIC:                             2323.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(8.205599479128068, 0.04194830055656455, 2.7561224748319266, 0.04177742749806321)
2.260588534879668
                            OLS Regression Results                            
Dep. Variable:           FR US Equity   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.412
Date:                Tue, 22 Nov 2022   Prob (F-statistic):             0.0173
Time:                        14:14:11   Log-Likelihood:                -1454.7
No. Observations:                 558   AIC:                             2917.
Df Residuals:                     554   BIC:                             2935.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(12.093759042673703, 0.007068786408188448, 4.0910215020879095, 0.006886527649244503)
2.533028949819745
                            OLS Regression Results                            
Dep. Variable:         APLE US Equity   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     19.41
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           5.60e-12
Time:                        14:14:11   Log-Likelihood:                -561.49
No. Observations:                 558   AIC:                             1131.
Df Residuals:                     554   BIC:                             1148.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

...

(1.7202492000660596, 0.6324413583264101, 0.5710664196483538, 0.6342474891834137)
2.024290111368523
                            OLS Regression Results                            
Dep. Variable:         ESRT US Equity   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     7.231
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           9.13e-05
Time:                        14:14:12   Log-Likelihood:                -549.05
No. Observations:                 558   AIC:                             1106.
Df Residuals:                     554   BIC:                             1123.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(7.357429657543602, 0.06133710227480035, 2.4674300231601514, 0.06126969566273196)
2.6501975927013555
                            OLS Regression Results                            
Dep. Variable:         GOOD US Equity   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.5697
Date:                Tue, 22 Nov 2022   Prob (F-statistic):              0.635
Time:                        14:14:12   Log-Likelihood:                -758.12
No. Observations:                 558   AIC:                             1524.
Df Residuals:                     554   BIC:                             1542.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

(30.45379081876477, 1.1076964963484209e-06, 10.660298453463446, 8.020896076522865e-07)
2.4834676951724624
                            OLS Regression Results                            
Dep. Variable:          FSP US Equity   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.3017
Date:                Tue, 22 Nov 2022   Prob (F-statistic):              0.824
Time:                        14:14:12   Log-Likelihood:                -291.13
No. Observations:                 558   AIC:                             590.3
Df Residuals:                     554   BIC:                             607.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

(1.484555603230325, 0.6858394490736084, 0.4926151421134979, 0.6875404969719381)
2.0540162508130355
                            OLS Regression Results                            
Dep. Variable:         WHLR US Equity   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.434
Date:                Tue, 22 Nov 2022   Prob (F-statistic):             0.0640
Time:                        14:14:12   Log-Likelihood:                -998.01
No. Observations:                 558   AIC:                             2004.
Df Residuals:                     554   BIC:                             2021.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


### Summary Results

Coefficients significantly lower and R2 significantly higher. Will most likely stick with the continous return model as it seems to produce the most credible results. 

## Coefficient means

In [14]:
Coef=pd.DataFrame(Coefficients) # Transforming coefficient list into datafrae
round(Coef["M"].mean(),4) # Mean market coefficient

0.6705

In [15]:
# Mean lag market coefficient
round(Coef["Mlag"].mean(),4)

0.0269

In [16]:
# Mean lead market coefficient
round(Coef["Mlead"].mean(),4)

0.0284

## Coefficient medians

In [22]:
# Median market coefficient
round(Coef["M"].median(),4)

0.4022

In [23]:
# Median market lag coefficient
round(Coef["Mlag"].median(),4)

0.021

In [19]:
# Median p-value of market coefficient
round(PV["M"].median(),4)

0.0002

## P-values

In [17]:
# Transforming list of p-values into a data frame
PV=pd.DataFrame(Pvalues)

In [18]:
# Mean p-value of market coefficient
round(PV["M"].mean(),4)

0.0924

In [26]:
round(PV["M"].median(),4)

0.0002

In [20]:
# # Mean p-value of lag market coefficient
PV["Mlag"].mean()

0.500760676305877

In [21]:
# Mean p-value of lead market coefficient
PV["Mlead"].mean()

0.4166952105207697

In [24]:
# Median lead coefficient
round(Coef["Mlead"].median(),4)

-0.001

## R-Squared

In [25]:
# Mean r-squared 
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.0349
dtype: float64

In [27]:
# Market coefficient mean standard deviation
STD=pd.DataFrame(STD)

round(STD["M"].median(),4)

0.1021

### Stationarity test

#### Just to be sure. Does not display non-stationarity

In [28]:
List=[]
for i in df:
    list=ts.adfuller(df[i])
    List.append(list)

In [29]:
List=pd.DataFrame(List)
round(List[1].mean(),4)

0.0049

## Direction of variables

In [30]:
Coef[(Coef["M"]<0)].count()

const    7
M        7
Mlag     7
Mlead    7
dtype: int64

In [31]:
d=Coef[(Coef["M"]>0)].count()

In [32]:
e=d["M"]/158

In [33]:
round(e,4)*100

95.57

In [34]:
round((1-e),4)*100

4.43

## Percentage significant

In [38]:
PV[(PV["M"]<0.1)].count()

const    132
M        132
Mlag     132
Mlead    132
dtype: int64

In [39]:
round(132/158,4)*100

83.54